In [1]:
%matplotlib qt
import numpy as np 
import matplotlib.pyplot as plt 

/home/jack/Software/Python/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Let's setup the initial parameters and the Kalman filter.

First we set the number of iteration steps, the initial position of the robot, and enable or disable movement of the robot. Next we setup the state transition matrices and the initial assumptions of the Kalman filter.

#### Initial State ####
The initial state (position) of the robot is modeled as a Gaussian with mean $x_0$ and covariance $\Sigma_0$.

#### State Transition Probability ###
The state transition probability $p(x_t\ |\ u_t,\ x_t-1)$ is a linear Gaussian function of the previous state, the current control input, and Gaussian noise:

\begin{equation}
x_t = A_tx_{t-1} + B_tu_t + \epsilon_t
\end{equation}

where $ A_t $ is an $n x n$ state transition matrix where $n$ is the dimension of the state vector $x_t$, which in this case is 2, and $B_t$ is an $n x m$ control matrix where $m$ is the dimension of the control vector $u_t$. $\epsilon_t$ is an n-dimensional Gaussian random vector that models the uncertainty introduced by the state transition. It's mean is zero and its covariance is $R_t$.

#### Measurement Probability ####
The measurement probability is a linear Gaussian function of the current state $x_t$ and Gaussian noise:

$$ z_t = C_tx_t + \delta_t $$

where $C_t$ is a matrix of size $kxn$ where $k$ is the dimension of the measurement vector $z_t$. The vector $\delta_t$ is the measurement noise modeled by a multivariate Gaussian with zero mean and covariance $Q_t$.

In [ ]:
# setup simulation parameters
n_iter = 100 # number of iterations
n = 2 # state vector dimension
m = 2 # control vector dimension
k = 2

# initial state Gaussian
x0 = np.array([-0.5],[0.5]) #Initial state vector
Sigma_0 = np.array([[1,1],[1,1]]) #Initial state covariance

# transition matrices
A_t = np.zeros((n,n)) # position matrix
B_t = np.zeros((n,m)) # control matrix
C_t = np.zeros((n,k)) # measturement matrix

# covariance matrices
R = np.array([[0.1, 0.1], 
              [0.1, 0.1]])# state transition noise covariance
Q = np.array([[1.0, 1.0],
              [1.0, 1.0]]) # measurement noise covariance

# initialize arrays to hold state, control, and measurement values (to plot later)
x = np.zeros((n,n_iter))	# array of actual position values
u = np.zeros((m, n_iter))    # array of control inputs
z = np.zeros((k, n_iter))	# measurement array

# initialize the posterior Gaussian
xhat = np.zeros((n,n_iter))	# posterior estimates of x
Sigma_hat = np.zeros((n_iter,n,n))	# posterior error estimates (covariance)

K = np.zeros(sz)	# Kalman gain or blending factor

# initial position
x[0] = x0

### Recursive Kalman Filter Algorithm ###
Now that the filter has been set up, we can define a function to actually perform the filter, then recursively apply the filter over the `n_iter` number of time steps.

#### Filter Inputs and Outputs ####
The recursive Kalman filter takes as inputs the previous time-step's state probability in the form of the the mean $\mu_{t-1}$ and the covariance $\Sigma_{t-1}$, as well as the control input at that time step $u_t$ and the measurement at that step $z_t$. The filter returns the state probability at that time step as the mean $\mu_t$ and covariance $\Sigma_t$. The mean and covariance returned at time $t$ then becomes the input to the filter at time $t+1$, the filter is applied again, and so on and so forth.

#### Predicted Belief (state probability update) ####
The filter starts by making a predicted belief of the state probability at time $t$ from the information provided about the state probability at time $t-1$, the control input to the state transition function from $t-1$ to $t$, and the measurement of the current state at $t$. As a reminder, the state transition function connects the state at $t-1$ to the current state at $t$ by:

$$ x_t = A_tx_{t-1} + B_tu_t + \epsilon_t $$

The Kalman filter predicts the mean and covariance of the state at $t$ in the following way:

$$ \bar{\mu_t} = A_t\mu_{t-1} + V_tu_t $$
$$ \bar{\Sigma_t} = A_t\Sigma_{t-1}A_t^T + R_t $$

Where the predicted belief of the state is represented by the predicted mean $\bar{\mu_t}$ and the predicted covariance $\bar{\Sigma_t}$, which describes a standard normal Gaussian probability distribution of the current predicted state, which in this case is the x-y location of a robot in a 2D grid.

In the first equation, the mean update, the $A_t$ and $B_t$ are the state transition and control matrices, respectively. $A_t$ effectively describes how the system changes on its own from time $t-1$ to $t$, while $B_t$ describes how a control input $u_t$ affects the state of the system from $t-1$ to $t$.

In the second equation, the covariance update considers the fact that states depend on previous states through the linear matrix $A_t$, which is multiplied twice into the covariance, since the covariance is a quadratic matrix.

#### Calculating the Kalman Gain ####
The next part of the recursive filter algorithm calculates the Kalman gain $K_t$, which specifies the degree to which the measurement $z_t$ is to be incorporated into the new state estimate.

$$ K_t = \frac{\bar{\Sigma_t}C_t^T}{ C_t\bar{\Sigma_t}C_t^T + Q_t} $$

As a reminder, the measurement $z_t$ is given by:

$$z_t = C_tx_t + \delta_t$$

where $\delta_t$ is a multivariate Gaussian with zero mean and covariance $Q_t$, so $Q_t$ describes the "spread" of the noise.